In [ ]:
# Random sample 500 lines from a file
# Generate n no. of generations per question
# Label the generations using the ground truth

In [1]:
import random
import numpy as np
import torch


def set_seeds(seed=42):
    # Set the Python seed for the random module
    random.seed(seed)
    
    # Set the seed for numpy
    np.random.seed(seed)
    
    # Set the seed for PyTorch on CPU and GPU
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multiple GPUs

    # Ensure deterministic algorithms for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(4291)


In [2]:
# Random sample 500 lines from a file
import random
import json
file_path = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/grade-school-math/grade_school_math/data/train.jsonl"

def read_jsonl():
    with open(file_path, "r") as f:
        lines = f.readlines()
        random.shuffle(lines)
        lines = lines[:500]
    
        lines = [json.loads(line) for line in lines]
        return lines

In [3]:
import transformers

# base_model = "/lustre/fsw/portfolios/llmservice/users/sghotra/models/pre_train/Meta-Llama-3.1-8B-Instruct"
base_model = "/lustre/fsw/portfolios/llmservice/users/sghotra/models/pre_train/Meta-Llama-3.1-8B"

def load_model():
    pipeline = transformers.pipeline(
        "text-generation", 
        model=base_model, 
        model_kwargs={"torch_dtype": torch.bfloat16}, 
        device_map="auto",
        batch_size=64
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(base_model, padding_side="left", truncation_side="left")
    return pipeline, tokenizer

/lustre/fsw/portfolios/llmservice/users/sghotra/installs/miniforge3/envs/adv_verif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = read_jsonl()
pipeline, tokenizer = load_model()

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.12s/it]


In [21]:
import re
from fractions import Fraction

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False


def extract_answer_from_response(completion, delimiter):
    text = completion.split(delimiter)
    if len(text) > 1:
        extract_ans = text[-1].strip()
        match = re.search(r'[\-+]?\d*[\.,/]?\d+', extract_ans)
        if match:
            if '/' in match.group():
                denominator = match.group().split('/')[1]
                numerator = match.group().split('/')[0]
                if is_number(denominator) and is_number(numerator):
                    if denominator == '0':
                        return round(float(numerator.replace(',', '')))
                    else:
                        frac = Fraction(match.group().replace(',', ''))
                        num_numerator = frac.numerator
                        num_denominator = frac.denominator
                        return round(float(num_numerator / num_denominator))
                else:
                    return None
            else:
                if float(match.group().replace(',', '')) == float('inf'):
                    return None
                return round(float(match.group().replace(',', '')))
        else:
            return None
    else:
        return None


def get_gt_labels(data):
    gt_labels = []
    for line in data:
        sample = line['answer']
        ans = int(extract_answer_from_response(sample, delimiter='#### '))
        gt_labels.append(ans)
    return gt_labels


def label_fraction(gt_labels, gen_data):
    total_none = 0
    total = 0
    macro_pos_count = 0
    for i, gens in enumerate(gen_data):
        gt = gt_labels[i]
        micro_pos_count = 0
        micro_total_count = 0
        wrong = 0
        none = 0
        for ind_gen in gens:
            ans = extract_answer_from_response(ind_gen, delimiter='The final answer is ')
            if ans is None:
                total_none += 1
                none += 1
                continue

            ans = int(ans)
            total += 1
            micro_total_count += 1

            if ans == gt:
                micro_pos_count += 1
                macro_pos_count += 1
            else:
                wrong += 1
        
        if micro_total_count == 0:
            print("all None")
        else:
            print(f"Micro pos/neg frac: {round(micro_pos_count/micro_total_count, 2)} {round(1 - (micro_pos_count/micro_total_count), 2)} {micro_total_count}/{len(gens)} wrong:{wrong} none:{none}")
    
    if total == 0:
        return
    print(f"Macro pos/neg frac: {macro_pos_count/total} {1 - (macro_pos_count/total)}")
    

In [10]:
inst_e1 = """Given the following problem, reason and give a final answer to the problem. Your response should end with \"The final answer is [answer]\" where [answer] is the response to the problem.
Problem:
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"
"""
inst_e1_s = """Solution:
Step 1: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.
The final answer is 6
"""

inst_e2 = """Problem:
If there are 3 cars in the parking lot and 2 more cars arrive, how manycars are in the parking lot?
"""

inst_e2_s = """Solution:
Step 1: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.
The final answer is 5
"""

inst_e3 = """Problem:
Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
"""

inst_e3_s = """Solution:
Step 1: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74.
Step 2: After eating 35, they had 74 - 35 = 39.
The final answer is 39
"""

inst_e4 = """Problem:
Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
"""

inst_e4_s = """Solution:
Step 1: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.
The final answer is 8
"""

inst_e5 = """Problem:
Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
"""

inst_e5_s = """Solution:
Step 1: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.
The final answer is 9
"""

inst_e6 = """Problem:
There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
"""

inst_e6_s = """Solution:
Step 1: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added.
Step 2: 9 + 20 is 29.
The final answer is 29
"""

inst_e7 = """Problem:
Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
"""

inst_e7_s = """Solution:
Step 1: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35.
Step 2: After losing 2 more, he had 35 - 2 = 33 golf balls.
The final answer is 33
"""

inst_e8 = """Problem:
Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
"""

inst_e8_s = """Solution:
Step 1: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars.
Step 2: So she has 23 - 15 dollars left. 23 - 15 is 8.
The final answer is 8
"""
q_template = """Problem:
{question}
"""

In [6]:
Prompt = """Given the following problem, reason and give a final answer to the problem. Your response should end with \"The final answer is [answer]\" where [answer] is the response to the problem.
Problem:
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"

Solution:
Step 1: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.
The final answer is 6


Problem:
If there are 3 cars in the parking lot and 2 more cars arrive, how manycars are in the parking lot?

Solution:
Step 1: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.
The final answer is 5


Problem:
Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?

Solution:
Step 1: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74.
Step 2: After eating 35, they had 74 - 35 = 39.
The final answer is 39


Problem:
Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?

Solution:
Step 1: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.
The final answer is 8


Problem:
Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?

Solution:
Step 1: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.
The final answer is 9


Problem:
There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?

Solution:
Step 1: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added.
Step 2: 9 + 20 is 29.
The final answer is 29


Problem:
Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?

Solution:
Step 1: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35.
Step 2: After losing 2 more, he had 35 - 2 = 33 golf balls.
The final answer is 33


Problem:
Olivia has $23. She bought five bagels for $3 each. How much money does she have left?

Solution:
Step 1: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars.
Step 2: So she has 23 - 15 dollars left. 23 - 15 is 8.
The final answer is 8


Problem:
"""

In [7]:
question = "Reese has been practicing piano for four hours every week. How many hours will he practice after five months?"
prompt_input = Prompt + question + "\nSolution:\n"
print(prompt_input)

Given the following problem, reason and give a final answer to the problem. Your response should end with "The final answer is [answer]" where [answer] is the response to the problem.
Problem:
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"

Solution:
Step 1: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.
The final answer is 6


Problem:
If there are 3 cars in the parking lot and 2 more cars arrive, how manycars are in the parking lot?

Solution:
Step 1: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.
The final answer is 5


Problem:
Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?

Solution:
Step 1: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74.
Step 2: After eating 35, 

In [8]:
# Generate n no. of generations per question
# Load the model
# Generation code

def generate_samples(question, n):
    # ques = q_template.format(question=question)

    prompt_input = Prompt + question + "\nSolution:\n"

    # messages = [
    #     ("Input": "user", "content": inst_e1),
    #     {"role": "assistant", "content": inst_e1_s},
    #     # {"role": "user", "content": inst_e2},
    #     # {"role": "assistant", "content": inst_e2_s},
    #     # {"role": "user", "content": inst_e3},
    #     # {"role": "assistant", "content": inst_e3_s},
    #     # {"role": "user", "content": ques},
    # ]
    outputs = pipeline(
        prompt_input,
        do_sample=True,
        num_return_sequences=n,
        # num_beams=5,
        temperature=1.0,
        top_p=0.95,
        max_new_tokens=256,
        stop_strings=["Problem:", "\n\n"], 
        tokenizer=tokenizer,
        return_full_text=False
    )
    return outputs  #[0]["generated_text"][-1]['content']

In [34]:
question = "Reese has been practicing piano for four hours every week. How many hours will he practice after five months?"
n = 4

output = generate_samples(question, n)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [35]:
for i in range(n):
    print(output[i]["generated_text"])
    print("\n")
    print("---------")

Step 1: He practiced for four hours a week. There are 5 weeks in one month.
Step 2: There are 5 months in total, so 5 x 5 = 25 weeks.
Step 3: So he practiced for 4 x 25 = 100 hours in 5 months.
The final answer is 100





---------
Step 1: Reese practices 4 hours per week. In 5 months there are 5 * 4 = 20 weeks. So, 20 weeks = 20 * 4 = 80 hours.
The final answer is 80





---------
Step 1: Reese practices for 4 hours each week. If there are 4 weeks in a month, then he would practice 4 * 4 = 16 hours in a month. 5 months x 4 = 20.
The final answer is 20





---------
Step 1: Reese practices for four hours a week. There are 4 x 5 = 20 weeks in five months.
Step 2: So 20 * 4 = 80 hours in total.
The final answer is 80





---------


In [19]:
gt_labels = get_gt_labels(data[:32])

In [17]:
gen_data = []
for i in range(32):
    outputs = generate_samples(data[i]['question'], 16)
    indv_gens = [x["generated_text"] for x in outputs]
    gen_data.append(indv_gens)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [20]:
label_fraction(gt_labels, gen_data)

Micro pos/neg frac: 0.38 0.62 16/16 wrong:10 none:0
Micro pos/neg frac: 0.73 0.27 15/16 wrong:4 none:1
Micro pos/neg frac: 0.94 0.06000000000000005 16/16 wrong:1 none:0
Micro pos/neg frac: 0.87 0.13 15/16 wrong:2 none:1
Micro pos/neg frac: 0.25 0.75 16/16 wrong:12 none:0
Micro pos/neg frac: 0.0 1.0 16/16 wrong:16 none:0
Micro pos/neg frac: 0.06 0.94 16/16 wrong:15 none:0
Micro pos/neg frac: 0.88 0.12 16/16 wrong:2 none:0
Micro pos/neg frac: 0.81 0.18999999999999995 16/16 wrong:3 none:0
Micro pos/neg frac: 0.62 0.38 16/16 wrong:6 none:0
Micro pos/neg frac: 0.31 0.69 16/16 wrong:11 none:0
Micro pos/neg frac: 0.13 0.87 15/16 wrong:13 none:1
Micro pos/neg frac: 0.81 0.18999999999999995 16/16 wrong:3 none:0
Micro pos/neg frac: 0.88 0.12 16/16 wrong:2 none:0
Micro pos/neg frac: 0.56 0.43999999999999995 16/16 wrong:7 none:0
Micro pos/neg frac: 0.4 0.6 15/16 wrong:9 none:1
Micro pos/neg frac: 0.81 0.18999999999999995 16/16 wrong:3 none:0
Micro pos/neg frac: 0.5 0.5 16/16 wrong:8 none:0
Micro p

In [22]:
for gens in gen_data:
    for g in gens:
        print(g)
    
    print("-------------")
    print("\n")

Step 1: Every week, he practices for 4 hours. Since he has been practicing for five months, he has practiced for 5 x 4 = 20 weeks.
Step 2: 4 hours every week for 20 weeks is 20 x 4 = 80 hours.
The final answer is 80



Step 1: Reese has been practicing for four hours each week. If he practices this way for five months, then he will practice 4 x 5 = 20 hours.
The final answer is 20



Step 1: Four hours every week will be 4 x 5 = 20.
Step 2: 20 hours after 5 weeks.
The final answer is 20



Step 1: He practices piano for 4 hours a week, and he will do this every week for 5 months. So there are 4 * 5 = 20 hours in total.
The final answer is 20



Step 1: Reese has been practicing 4 hours a week for 5 months. Each month has 4 weeks, so 4 * 5 = 20 weeks. 20 weeks * 4 hours each = 80 hours.
The final answer is 80



Step 1: He has practiced for 4 hours for 1 week. After 5 months there will be 5 x 4 = 20 weeks.
Step 2: So he has practiced for 4 x 20 = 80 hours. The final answer is 80



Step

In [15]:
for i in range(5):
    outputs = generate_samples(data[i]['question'], 4)
    indv_gens = [x["generated_text"][-1]['content'] for x in outputs]
    for g in indv_gens:
        print(g)
        print("\n")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/lustre/fsw/portfolios/llmservice/users/sghotra/installs/miniforge3/envs/adv_verif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Solution:
Step 1: There are 4 weeks in a month. So in 5 months, there are 5 * 4 = 20 weeks.
Step 2: Reese practices 4 hours every week. So in 20 weeks, he will practice 20 * 4 = 80 hours.
The final answer is 80


Solution:
Step 1: There are 4 weeks in a month. So in 5 months, there are 5 * 4 = 20 weeks.
Step 2: Reese practices 4 hours every week. So in 20 weeks, he will practice 4 * 20 = 80 hours.
The final answer is 80


Solution:
Step 1: There are 4 weeks in a month. So in 5 months, there are 5 * 4 = 20 weeks.
Step 2: Reese practices 4 hours a week. So in 20 weeks, he will practice 20 * 4 = 80 hours.
The final answer is 80


Solution:
Step 1: There are 4 weeks in a month. So in 5 months, there are 5 * 4 = 20 weeks.
Step 2: Reese practices 4 hours per week. So in 20 weeks, he will practice 20 * 4 = 80 hours.
The final answer is 80




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Solution:
Step 1: Asha borrowed $20 from her brother, $40 from her father, $30 from her mother, and was gifted $70 by her granny. So the total amount of money she received is 20 + 40 + 30 + 70 = 160.
Step 2: She also had $100 in savings. So the total amount of money she has is 160 + 100 = 260.
Step 3: She spent 3/4 of the money. 3/4 of 260 is (3/4) * 260 = 195.
Step 4: To find out how much money she has left, we need to subtract the amount she spent from the total amount. 260 - 195 = 65.
The final answer is 65


Solution:
Step 1: Asha borrowed $20 from her brother, $40 from her father, $30 from her mother, and was gifted $70 by her granny. So the total amount of money she received is 20 + 40 + 30 + 70 = 160.
Step 2: She also had $100 in savings. So the total amount of money she has is 160 + 100 = 260.
Step 3: She spent 3/4 of the money. 3/4 of 260 is (3/4) * 260 = 195.
Step 4: To find out how much money she has left, we need to subtract the money she spent from the total amount of mone

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Solution:
Step 1: To get rid of each grass stain, Bill needs to soak his clothes for 4 minutes. Since there are 3 grass stains, he needs to soak his clothes for 3 * 4 = 12 minutes for the grass stains.
Step 2: To get rid of the marinara stain, Bill needs to soak his clothes for 7 minutes.
Step 3: In total, Bill needs to soak his clothes for 12 + 7 = 19 minutes.
The final answer is 19


Solution:
Step 1: To get rid of each grass stain, Bill needs to soak his clothes for 4 minutes. Since there are 3 grass stains, he needs to soak his clothes for 3 * 4 = 12 minutes for the grass stains.
Step 2: To get rid of the marinara stain, Bill needs to soak his clothes for 7 minutes.
Step 3: In total, Bill needs to soak his clothes for 12 (grass stains) + 7 (marinara stain) = 19 minutes.
The final answer is 19


Solution:
Step 1: To get rid of each grass stain, Bill needs to soak his clothes for 4 minutes. Since there are 3 grass stains, he needs 3 * 4 = 12 minutes for the grass stains.
Step 2: To g

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Solution:
Step 1: Tom drives 75 miles per day. In 10 days, he will drive 75 * 10 = 750 miles.
Step 2: His car gets 50 miles to the gallon. To drive 750 miles, he will need 750 / 50 = 15 gallons of gas.
Step 3: Gas costs $3 per gallon. For 15 gallons, he will spend 15 * $3 = $45.
The final answer is $45


Solution:
Step 1: Tom drives 75 miles per day. In 10 days, he will drive 75 * 10 = 750 miles.
Step 2: His car gets 50 miles to the gallon. To drive 750 miles, he will need 750 / 50 = 15 gallons of gas.
Step 3: Gas costs $3 per gallon. For 15 gallons, he will spend 15 * 3 = $45.
The final answer is 45


Solution:
Step 1: Tom drives 75 miles per day. In 10 days, he will drive 75 * 10 = 750 miles.
Step 2: His car gets 50 miles to the gallon. To drive 750 miles, he will need 750 / 50 = 15 gallons of gas.
Step 3: Gas costs $3 per gallon. So 15 gallons will cost 15 * $3 = $45.
The final answer is $45


Solution:
Step 1: Tom drives 75 miles per day. In 10 days, he will drive 75 * 10 = 750 mil